In [ ]:
import numpy as np # creating a short cut to acces numpy faster in future code
import matplotlib.pyplot as plt #importing matplotlib tool as a shortcut to be able to plot my conclusions. 
from numpy import loadtxt #importing a usefull tool from numpy

def read(file): #defining a new function to read in external files
    data = loadtxt(file) #calling the extracted file data
    data_new = data.transpose() # transposing the values, so I only got 3 rows of data and can acces it easier
    time = data_new[0] #the first row in this transposed data is the time the observations were made 
    flux = data_new[1] #the second row is the flux of the star
    error = data_new[2] #the last row is the error for the y-values(flux)
    
    return time, flux, error #returning three lists containing the unpacked values





files = ["transit_1.136-1.155micr.txt",
"transit_1.155-1.173micr.txt",
"transit_1.173-1.192micr.txt",

"transit_1.192-1.210micr.txt",

"transit_1.210-1.229micr.txt",

"transit_1.229-1.247micr.txt",

"transit_1.247-1.266micr.txt",

"transit_1.266-1.284micr.txt",

"transit_1.284-1.303micr.txt",

"transit_1.303-1.321micr.txt",

"transit_1.321-1.340micr.txt",

"transit_1.340-1.358micr.txt",

"transit_1.358-1.377micr.txt",

"transit_1.377-1.395micr.txt",

"transit_1.395-1.414micr.txt",

"transit_1.414-1.432micr.txt",

"transit_1.432-1.451micr.txt",

"transit_1.451-1.469micr.txt",

"transit_1.469-1.488micr.txt",

"transit_1.488-1.506micr.txt",

"transit_1.506-1.525micr.txt",

"transit_1.525-1.543micr.txt",

"transit_1.543-1.562micr.txt",

"transit_1.562-1.580micr.txt"]
import batman  
from scipy.optimize import minimize
import math
import emcee
rp = []

error_rp = []

In [ ]:
for i in range(len(files)): #creating a big for loop to compute the emcee values for all the 24 datasets seperately
    flux_bat = []
    time = []
    flux = []
    error = []
    
    time1, flux1, error1 = read(files[i])
    
    time.append(time1)
    flux.append(flux1)
    error.append(error1)
    
    time = np.concatenate(time)
    flux = np.concatenate(flux)
    flux = flux/1.0076100701903976
    error = np.concatenate(error)
    
    t = np.linspace(time[0], time[-1], len(time) )
    import batman                         #importing the batman module to help plotting the transit curve

    params = batman.TransitParams()       #object to store transit parameters
    params.t0 =  2457424.88307                  #time of inferior conjunction
    params.per = 1.2749255                       #orbital period
    params.rp = 0.121                       #planet radius (in units of stellar radii)
    params.a = 3.7                        #semi-major axis (in units of stellar radii)
    params.inc = 89                      #orbital inclination (in degrees)
    params.ecc = 0.                       #eccentricity
    params.w = 0.                        #longitude of periastron (in degrees)
    params.limb_dark = "nonlinear"        #limb darkening model
    params.u = [0.5, 0.1, 0.1, -0.1]      #limb darkening coefficients [u1, u2, u3, u4]


    m = batman.TransitModel(params, t)    #initializes model
    flux_bat = m.light_curve(params)          #calculates light curve
 
    plt.plot(time, flux,"o")
    plt.errorbar(time, flux, fmt='.', yerr=error) #plotting the errorbar
    plt.plot(t, flux_bat)
    plt.title(files[i], fontsize=40)
    plt.xlabel('Time in Days', fontsize=20)
    plt.ylabel('Flux ', fontsize=20)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.show() 
    
    def fun(x0,a , b, c): #fixing all values and only letting the rp/rs parameter vary
        params = batman.TransitParams()  
        params.t0 = 2457424.88307                        #time of inferior conjunction
        params.per = 1.2749255                       #orbital period
        params.rp = x0[0]                       #planet radius (in units of stellar radii)
        params.a = 3.631                       #semi-major axis (in units of stellar radii)
        params.inc = 87.903                      #orbital inclination (in degrees)
        params.ecc = 0.                       #eccentricity
        params.w = 0.                       #longitude of periastron (in degrees)
        params.limb_dark = "nonlinear"        #limb darkening model
        params.u = [0.5,0.1,0.1,-0.1]
    
    
    
    
        flux_bat = m.light_curve(params)
        if  0.0 < x0[0]   :
            return sum(((b-flux_bat)/c)**2)
        else:
             return float('inf')
    params = batman.TransitParams() 
    
    x0 = [0.121 ]
    
    res = minimize(fun, x0, args = (time, flux, error), method='Nelder-Mead', tol=1e-6 )
    x = res.x.tolist()
    theta = x
    x.insert(0,2457424.88307)
    x.insert(1,1.2749255)
    true = np.asarray(x)
    
    def log_likelihood(theta, time_fun, flux_fun, error_fun):
    
    
        params = batman.TransitParams()       #object to store transit parameters
        params.t0 = theta[0]                        #time of inferior conjunction
        params.per = theta[1]                 #orbital period
        params.rp = theta[2]                      #planet radius (in units of stellar radii)
        params.a =  3.631                      #semi-major axis (in units of stellar radii)
        params.inc = 87.903                      #orbital inclination (in degrees)
        params.ecc = 0.                       #eccentricity
        params.w = 0.                       #longitude of periastron (in degrees)
        params.limb_dark = "nonlinear"        #limb darkening model
        params.u = [0.5,0.1,0.1,-0.1]  
        model = m.light_curve(params)
    
    
   
        return -0.5 * (np.sum(((flux_fun - model) / error_fun)**2)+np.sum(((2457424.88307 - theta[0]) / 0.00011)**2)+ np.sum(((1.2749255-theta[1]) / 0.00000025)**2))

    def log_prior(theta):
    
        if  0.0 < theta[1]  and 0.0 < theta[2] < 1  :
            return 0.0
  
        
        return -np.inf
    
    def log_probability(theta, time_fun, flux_fun, error_fun):
        lp = log_prior(theta)
    
        if not np.isfinite(lp):
            return -np.inf
        return lp + log_likelihood(theta, time_fun, flux_fun, error_fun)
    
    
    
    ndim = 3
    nwalkers = 50
    pos = true + 1e-4 * np.random.randn(50, 3)
    nwalkers, ndim = pos.shape
    sampler = emcee.EnsembleSampler(
        nwalkers, ndim, log_probability, args=( time, flux, error),
    moves=emcee.moves.StretchMove())
    sampler.run_mcmc(pos , 6000, progress=True);
    
    fig, axes = plt.subplots(3, figsize=(10, 7), sharex=True)
    samples = sampler.get_chain(discard= 1000)

    for i in range(ndim):
        ax = axes[i]
        ax.plot(samples[:, :, i], "k", alpha=0.3)
        ax.set_xlim(0, len(samples))
        
    labels = ["Time" "" "of" "Inferior" "Conjunction","Orbital" "Period","Planet"" Radius"]
    import corner

    flat_samples = sampler.get_chain( flat=True, discard = 1000)
    fig = corner.corner(
        flat_samples,   truths= true, labels = labels)
    
    print(files[i])
    from IPython.display import display, Math
    results = []
    result_uncertainty = []
    for i in range(ndim):
        mcmc = np.percentile(flat_samples[:, i], [16, 50, 84])
        q = np.diff(mcmc)
        txt = "\mathrm{{{3}}} = {0:.3f}_{{-{1:.3f}}}^{{{2:.3f}}}"
        txt = txt.format(mcmc[1], q[0], q[1], labels[i])
        display(Math(txt))
        result_uncertainty.append([ mcmc[1], q[0], q[1]])
        results.append(mcmc[1])
        
    rp.append(results[2])
    
    error_rp.append(result_uncertainty[2])

In [ ]:
wavelengths = [[1.136,1.155],
[1.155,1.173],
[1.173,1.192],
[1.192,1.210],
[1.210,1.229],
[1.229,1.247],
[1.247,1.266],
[1.266,1.284],
[1.284,1.303],
[1.303,1.321],
[1.321,1.340],
[1.340,1.358],
[1.358,1.377],
[1.377,1.395],
[1.395,1.414],
[1.414,1.432],
[1.432,1.451],
[1.451,1.469],
[1.469,1.488],
[1.488,1.506],
[1.506,1.525],
[1.525,1.543],
[1.543,1.562],
[1.562,1.580]]
wavelength = []

for i in range(len(wavelengths)): #creating a average wavelength for every wavelength interval
    
    wavelength.append((wavelengths[i][0]+wavelengths[i][1])/2)
er_up = []
er_down = []

In [ ]:
for i in range(24): #extracting the negative and positive error for the rp/rs value
    er_up.append(error_rp[i][1])
    er_down.append(error_rp[i][2])

In [ ]:
wave = [] #creating an error bar with the width of the corresponding wavelength interval

for i in range(len(wavelengths)):
    wave.append((wavelengths[i][1]-wavelengths[i][0])/2)


In [ ]:
error_total = [er_down, er_up] #plotting the transit spectrum

plt.plot(wavelength,rp, 'o' )
plt.errorbar(wavelength, rp, fmt='.', yerr= error_total)
plt.errorbar(wavelength, rp, fmt='.', xerr=wave)
plt.xlabel('Wavelength in Micrometer', fontsize=20)
plt.ylabel('Rp/Rs ', fontsize=20)


In [ ]:
from astropy.io import ascii
data = ascii.read("Evans Data.txt")  #reading in the observed rp/rs data from Evans et al.
for i in range(4):
    del(data[0])
for i in range(3):
    del(data[-1])


In [ ]:
wavelength_evans = []
value_evans = [] #creating lists for the Evans et al. values, errors and wavelength intervals
er_up_evans = []
er_down_evans = []
wave_evans = []
for i in range(len(data)):
    wavelength_evans.append((data[i][0]+abs(data[i][1]))/2)
    value_evans.append(data[i][2])
    er_up_evans.append(data[i][4])
    er_down_evans.append(data[i][3])
    wave_evans.append((abs(data[i][1])-data[i][0])/2)


In [ ]:
import re #making the values compatible for plotting
for i in range(len(value_evans)):
    value_evans[i] = re.sub('[^\d\.]', '', value_evans[i])
    er_up_evans[i] = re.sub('[^\d\.]', '', er_up_evans[i])
    er_down_evans[i] = re.sub('[^\d\.]', '', er_down_evans[i])
    value_evans[i] = float(value_evans[i])
    er_up_evans[i] = float(er_up_evans[i])
    er_down_evans[i] = float(er_down_evans[i])


In [ ]:
error_total_evans = [er_down_evans, er_up_evans] #plotting both transit spectra against each other
plt.plot(wavelength,rp, 'o', label = "Computed Values")
plt.errorbar(wavelength, rp, fmt='.', yerr= error_total)
plt.errorbar(wavelength, rp, fmt='.', xerr=wave)
plt.plot(wavelength_evans,value_evans, 'o', label = "Evans et al. Values" )
plt.errorbar(wavelength_evans, value_evans, fmt='.', yerr= error_total_evans)
plt.errorbar(wavelength_evans, value_evans, fmt='.', xerr=wave_evans)

plt.xlabel('Wavelength in Micrometer', fontsize=20)
plt.ylabel('Rp/Rs ', fontsize=20)
plt.legend()

In [ ]:
print("rp", "     value_evans  ","er_up ", "     er_down", "    difference") #printing a table with all the values compared
for i in range(len(rp)):
    print(round(rp[i],5),"|",value_evans[i],"|", round(er_up[i],9),"|", round(er_down[i],9),"|", round(value_evans[i]-rp[i],6))